###### Imports and Settings

In [1]:
import pandas as pd
#import geopandas as gpd
import numpy as np
import requests
import io
import pickle
import matplotlib.pyplot as plt
from collections import deque
from functools import reduce
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)

###### Functions

In [2]:
def percent(x, y):
    return round((x/y)*100, 2)

###### API Key

In [6]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)    
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

## Read in Data Guide

The "head" should never be more than 2 variables, and the "tail" never more than 2. Since we can pull 50 variables at once this means that we can systematically program in 46 variables for each pull, so that's:
+ dg1: ID's 1 - 46  
+ dg2: ID's 47-92 
+ dg3: ID's 93-138  

In [61]:
dataguide = pd.read_csv('../data guides/dataguide.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [62]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
dg2 = dataguide[dataguide['ID'].between(47, 92)]
dg3 = dataguide[dataguide['ID'].between(93, 138)]

In [63]:
#ONE
# ONE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg1['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg1['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:037"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
one=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
print('Okay Finished')

Okay Finished


In [64]:
#TWO
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg2['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg2['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:037"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
two=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
print('Okay Finished')

Okay Finished


In [65]:
#THREE
# listparams
head1 = 'NAME'
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
#make variables list
var_list = list(dg3['ACS Variable'])
#add stuff to variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
var_list = list(var_list)
#make columns list
col_list = list(dg3['Column Name'])
#add stuff to columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
col_list = list(col_list)
url_str= 'https://api.census.gov/data/2020/acs/acs5?key='+api_key
predicates= {}
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:037"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
last=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
print('Okay Finished')

Okay Finished


## Clean and Join

In [66]:
one = one.drop(columns = ['StateFIPS','GeoFIPS'])
two = two.drop(columns = ['NAME','StateFIPS','GeoFIPS'])
last = last.drop(columns = 'NAME')

In [67]:
dfs = [one, two, last]
data = reduce(lambda  left,right: pd.merge(left,right,on=['GEO_ID'],
                                            how='outer'), dfs)

In [68]:
data = data.set_index(['NAME', 'GEO_ID', 'StateFIPS', 'GeoFIPS'])
numcols = list(data.columns)
data[numcols] = data[numcols].astype(float)
data = data.reset_index()

## Calculations

#### Total Age Groups

In [69]:
data['Total Population'] = data['pop']
over65 = [data['age_m_65to66'],data['age_m_67to69'],data['age_m_70to74'],data['age_m_75to79'],data['age_m_80to84'],data['age_m_85+'],data['age_f_65to66'],
          data['age_f_67to69'],data['age_f_70to74'],data['age_f_75to79'],data['age_f_80to84'],data['age_f_85+']]
data['Total Population Over Age 65'] = sum(over65)
data['Percent of Population Over Age 65'] = percent(data['Total Population Over Age 65'], data['Total Population'])

cols = ['pop','agebysex_total_series','age_total_male','age_m_u5','age_m_5to9','age_m_10to14','age_m_15to17','age_m_18to19','age_m_20','age_m_21','age_m_22to24',
        'age_m_25to29','age_m_30to34','age_m_35to39','age_m_40to44','age_m_45to49','age_m_50to54','age_m_55to59','age_m_60to61','age_m_62to64','age_m_65to66',
        'age_m_67to69','age_m_70to74','age_m_75to79','age_m_80to84','age_m_85+','age_total_female','age_f_u5','age_f_5to9','age_f_10to14','age_f_15to17',
        'age_f_18to19','age_f_20','age_f_21','age_f_22to24','age_f_25to29','age_f_30to34','age_f_35to39','age_f_40to44','age_f_45to49','age_f_50to54','age_f_55to59',
        'age_f_60to61','age_f_62to64','age_f_65to66','age_f_67to69','age_f_70to74','age_f_75to79','age_f_80to84','age_f_85+']
data = data.drop(columns = cols)

#### Tenure and Cost Burden

In [70]:
renters65up = data['tenure_renter_65to74']+data['tenure_renter_75to84']+data['tenure_renter_85+']
owners65up = data['tenure_owner_65to74']+data['tenure_owner_75to84']+data['tenure_owner_85+']
hh65older = [data['tenure_renter_65to74'],data['tenure_renter_75to84'],data['tenure_renter_85+'],data['tenure_owner_65to74'],
             data['tenure_owner_75to84'],data['tenure_owner_85+']]
householders65up = sum(hh65older)
allcostburdeno65 = [data['housing_65+30to34.9%_ownercosts'],data['housing_65+35%ormore_ownercosts'],data['housing_65+30to34.9%_grossrent'],
                    data['housing_65+35%ormore_grossrent']]
data['Householders 65 and Older Experiencing Cost Burden'] = sum(allcostburdeno65)
data['Percent of Householders 65 and Older Experiencing Cost Burden'] = percent(data['Householders 65 and Older Experiencing Cost Burden'], householders65up)
data['Renters 65 and Older Experiencing Cost Burden'] = data['housing_65+30to34.9%_grossrent']+data['housing_65+35%ormore_grossrent']
data['Percent of Renters 65 and Older Experiencing Cost Burden'] = percent(data['Renters 65 and Older Experiencing Cost Burden'],renters65up)
data['Homeowners 65 and Older Experiencing Cost Burden'] = data['housing_65+30to34.9%_ownercosts']+data['housing_65+35%ormore_ownercosts']
data['Percent of Homeowners 65 and Older Experiencing Cost Burden'] = percent(data['Homeowners 65 and Older Experiencing Cost Burden'],owners65up)

cols = ['tenure_total_agehh_series','tenure_allowneroccupied','tenure_owner_55to59','tenure_owner_60to64','tenure_owner_65to74','tenure_owner_75to84',
        'tenure_owner_85+','tenure_allrenteroccupied','tenure_renter_55to59','tenure_renter_60to64','tenure_renter_65to74','tenure_renter_75to84',
        'tenure_renter_85+','housing_total_ownercosts','housing_65+total_ownercosts','housing_65+30to34.9%_ownercosts','housing_65+35%ormore_ownercosts',
        'housing_total_grossrent','housing_65+total_grossrent','housing_65+30to34.9%_grossrent','housing_65+35%ormore_grossrent']
data = data.drop(columns = cols)

#### Income

In [71]:
data['Median Household Income Householder 65 and Older'] = data['hhincome_65+_median']
lowincome = [data['hhincome_65+_lessthan10000'], data['hhincome_65+_10000to14999'],data['hhincome_65+_15000to19999'],data['hhincome_65+_20000to24999'],
            data['hhincome_65+_25000to29999'],data['hhincome_65+_30000to34999'],data['hhincome_65+_35000to39999'],data['hhincome_65+_40000to44999'],
            data['hhincome_65+_45000to49999']]
data['Household Income Below $50,000 Householder 65 and Older'] = sum(lowincome)

cols = ['hhincome_65+_median','hhincome_totalhh65+_series','hhincome_65+_lessthan10000','hhincome_65+_10000to14999','hhincome_65+_15000to19999',
        'hhincome_65+_20000to24999','hhincome_65+_25000to29999','hhincome_65+_30000to34999','hhincome_65+_35000to39999','hhincome_65+_40000to44999',
        'hhincome_65+_45000to49999','hhincome_65+_50000to55999','hhincome_65+_60000to74999','hhincome_65+_75000to99999','hhincome_65+_100000to124999',
        'hhincome_65+_125000to149999','hhincome_65+_150000to199999','hhincome_65+_200000+']
data = data.drop(columns = cols)

#### Poverty

In [72]:
poverty65up = [data['poverty_m65to74'],data['poverty_m75+'],data['poverty_f65to74'],data['poverty_f75+']]
data['Population 65 and Older Living Below Poverty Line'] = sum(poverty65up)

cols = ['poverty_total_series','poverty_totalbelowpoverty','poverty_m65to74','poverty_m75+','poverty_f65to74','poverty_f75+']
data = data.drop(columns = cols)

### Prepare for Export and Export

In [74]:
data = data.drop(columns = ['GEO_ID','StateFIPS','GeoFIPS','Total Population','Percent of Population Over Age 65',
                            'Percent of Householders 65 and Older Experiencing Cost Burden','Renters 65 and Older Experiencing Cost Burden',
                           'Homeowners 65 and Older Experiencing Cost Burden'])

In [75]:
data.head()

,NAME,Total Population Over Age 65,Householders 65 and Older Experiencing Cost Burden,Percent of Renters 65 and Older Experiencing Cost Burden,Percent of Homeowners 65 and Older Experiencing Cost Burden,Median Household Income Householder 65 and Older,"Household Income Below $50,000 Householder 65 and Older",Population 65 and Older Living Below Poverty Line
0,"Davidson County, Tennessee",84748.0,17682.0,52.8,24.21,49128.0,28533.0,8110.0


In [76]:
data.to_csv('../data/65olderdavidsoncounty.csv', index = False)